Thursday, November 16, 2023

https://huggingface.co/TheBloke/CodeLlama-34B-Instruct-GPTQ

Let's see if we can get this to run locally, shall we ... 

I am trying this model because it has the most downloads of all his models, AND of all the huggingface models that are 'code' related.

Wow! This actually works on the 4090! I am stunned! 


In [ ]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--CodeLlama-34B-Instruct-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 18.3GB to /home/rob/Data3/huggingface/transformers


In [1]:
# You need this to just target the 4090, othewise the 34B model does NOT load.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Nope! This does not load to the 4090! Too damn big!
model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


In [3]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_name_or_path = "TheBloke/CodeLlama-34B-Instruct-GPTQ"
# # To use a different branch, change revision
# # For example: revision="main"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
{prompt}
[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])

# 22.5s




*** Generate:
<s> [INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

Here is an example of a code that obeys the constraints and passes the example test cases:
```
def tell_me_about_ai(text):
    # Remove any unnecessary characters from the input text
    text = text.lower().strip()

    # Check if the input text contains the word "ai"
    if "ai" in text:
        # If the input text contains the word "ai", return a statement about AI
        return "AI is a rapidly growing field that involves developing computer systems and algorithms that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision making, and language translation."
    else:
        # If the input text does not contain the word "ai", return a statement about the input text
        return "The input text does not contain the word

In [4]:
# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

# 6.9s

*** Pipeline:
[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Tell me about AI
[/INST]

Using Python, we can use the `random` module to generate a random integer between 0 and 100, inclusive. We can then use this number to determine which of the three statements is true:
```
import random

# Generate a random integer between 0 and 100
x = random.randint(0, 100)

# Determine which statement is true based on x
if x <= 33:
    print("AI is a tool used by humans.")
elif x > 33 and x <= 66:
    print("AI is a human who uses tools.")
else:
    print("AI is a tool that uses other tools.")
```
